In [9]:
from datetime import datetime

In [10]:
# edges are encoded at 8 bit bytes. The first bit encodes flipped or not. The last 4 bits encode slice (0, 1, 2) and number within the slice (0, 1, 2, 3)

edges   = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]
corners = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07]
cube    = bytes(edges + corners)



In [11]:
def generate_new(members, generators, subgroup_mask):
    
    members_list = [[membercube[i] & subgroup_mask[i] for i in range(0,20)] for membercube in members.values()]
   
    candidates, new_members = dict(), dict()
    
    for name, membercube in members.items():
        candidates.update({name + n: bytes([membercube[0x0F & generator[i]] ^ (generator[i] & 0xF0)  for i in range(0,12)] +
                                      [(membercube[12 + (0x0F & generator[j])] & 0x0F) + (((membercube[12 + (0x0F & generator[j])] & 0xF0) + (generator[j] & 0xF0))%0xC0)    for j in range(12, 20)])
                           for n, generator in generators.items()} )
    
      
    for name, testcube in candidates.items():
        candidate = [[testcube[i] & subgroup_mask[i] for i in range(0,20)] for cube in members.values()]
        if candidate not in members_list:            
            new_members[name] = testcube
            members_list.append(candidate)
            
    return new_members
    
    

In [12]:
# Define base permutations (clockwise quarter turns in each face - cubies numbered from 0
moves = {'R':  bytes([0x00, 0x01, 0x06, 0x07, 0x04, 0x05, 0x03, 0x02, 0x08, 0x09, 0x0A, 0x0B]+ [0x00, 0x01, 0x07, 0x06, 0x04, 0x05, 0x02, 0x03]),
         'L':  bytes([0x04, 0x05, 0x02, 0x03, 0x01, 0x00, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]+ [0x05, 0x04, 0x02, 0x03, 0x00, 0x01, 0x06, 0x07]),
         'F':  bytes([0x00, 0x01, 0x02, 0x03, 0x04, 0x09, 0x08, 0x07, 0x05, 0x06, 0x0A, 0x0B]+ [0x00, 0x86, 0x02, 0x84, 0x41, 0x05, 0x43, 0x07]),
         'B':  bytes([0x00, 0x01, 0x02, 0x03, 0x0B, 0x05, 0x06, 0x0A, 0x08, 0x09, 0x04, 0x07]+ [0x87, 0x01, 0x85, 0x03, 0x04, 0x40, 0x06, 0x42]),
         'U':  bytes([0x88, 0x01, 0x8B, 0x03, 0x04, 0x05, 0x06, 0x07, 0x82, 0x09, 0x0A, 0x80]+ [0x44, 0x01, 0x02, 0x47, 0x83, 0x05, 0x06, 0x80]),
         'D':  bytes([0x00, 0x8A, 0x02, 0x89, 0x04, 0x05, 0x06, 0x07, 0x08, 0x81, 0x83, 0x0B]+ [0x00, 0x45, 0x46, 0x03, 0x04, 0x82, 0x81, 0x07])}
#         'UU': bytes([2, 1, 0, 3, 4, 5, 6, 7, 11, 9, 10, 8]+ [3, 1, 2, 0, 7, 5, 6, 4]),
#         'DD': bytes([0, 3, 2, 1, 4, 5, 6, 7, 8, 10, 9, 11]+ [0, 2, 1, 3, 4, 6, 5, 7])}

In [16]:
start = datetime.now()
#group 1 subgroup_masks = [[0x80], [0x00]]
#group 2 subgroup_masks = [[0x04], [0xC0]]
subgroup_masks = [[0x80], [0x00]]
bit_mask = bytes(subgroup_masks[0] * 12 + subgroup_masks[1] * 8 )
new, members = 1, {'': cube}
loop = 0
while new > 0 :
    new_members = generate_new(members, moves, bit_mask)
    new = len(new_members)
    print(str((datetime.now() - start)), len(members), len(new_members))
    members.update(new_members)


    loop += 1
   

0:00:00 1 3
0:00:00.000992 4 12
0:00:00.002005 13 60
0:00:00.033107 61 250
0:00:00.742878 251 794
0:00:06.167488 795 1628
0:00:28.790191 1629 2031
0:01:03.157749 2032 2048
0:01:38.252589 2049 2048
0:02:13.105222 2049 2048
0:02:48.753468 2049 2048


KeyboardInterrupt: 